In [1]:
#from src.utils.builder import build_trainer, build_config
from src.utils.utils import *
from src.utils.builder import *
from src.utils.fileio import *
from src.models.modules.image_encoder import *
from functools import partial 
from arguments import args
from time import time
from PIL import Image
import numpy as np
import torchvision.transforms as transforms 
import torch 
import torch.nn as nn
import timm 
from thop import profile

In [2]:
from src.models.model_zoo.masked_vision_model import *
from src.models.modules.masked_vision_layers import *
from src.models.model_zoo.vector_quantized_mae import VQMaskedImageAutoEncoder

In [3]:
class Args:
    default_config_path= './configs/default.yaml'
    model_config_path='./configs/models/masked_image.yaml'
    dataset_config_path='./configs/datasets/stereo_mim.yaml'
    user_config_path='./configs/user/sample.yaml'
    local_rank=None
    opts=None

args=Args()

In [4]:
# --------------- configure initialisers -----------------
setup_imports()

config = build_config(args)

# setup aws env variables:
fileio_client = FileIOClient(config)

# check correct config loaded;
print(config.model_config.name)
#print(config.dataset_config.usable_columns)

Global seed set to 42424242


THIS VERSION OF THE CODE SUPPORTS LOCAL FILE I/O ONLY! FOR CLOUD CLIENTS CONTACT THE MAINTAINER
output dir for saving files: "/data/samyakhtukra/mim/mae_stereo_test/230210-192249/train_outputs" created!
masked_image_autoencoder


In [5]:
#model = MaskedImageAutoEncoderMSGGAN(config)
img_size = config.dataset_config.preprocess.vision_transforms.params.Resize.size
patch_size = config.model_config.image_encoder.patch_size
in_channels = config.model_config.image_encoder.in_channels
embed_dim = config.model_config.image_encoder.embed_dim
norm_layer_arg= config.model_config.norm_layer_arg
        
if norm_layer_arg=='partial':
    norm_layer = partial(nn.LayerNorm, eps=1e-6)
    print('using partial layer norm')
else:
    norm_layer = nn.LayerNorm

patch_embed = PatchEmbed(img_size, patch_size, in_channels, embed_dim)

using partial layer norm


In [6]:
model = MaskedImageAutoEncoder(config)

for this work well, make sure inputs are not normalised!
using partial layer norm


In [7]:
print(model)

MaskedImageAutoEncoder(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 32), stride=(16, 32))
  )
  (encoder): MAEEncoder(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 32), stride=(16, 32))
    )
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (drop1): Dropout(p=0.0, inplace=False)
          (fc2): Linear(in_features

In [9]:
# create random input tensor:
left_image = torch.randn((8,3,224,448))
right_image = torch.randn((8,3,224,448))
# ----------------- model inference ----------------
output = model(left_image, right_image)

In [10]:
print(type(output))

<class 'tuple'>


In [11]:
print(output[0][0].size(), output[0][1].size())

torch.Size([8, 196, 1536]) torch.Size([8, 196, 1536])
